In [ ]:
# Plant Disease Image Classifier - CNN

# Set seeds for reproducability
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# !pip install kaggle

# Provide the full path to your kaggle.json file if it's not in the current directory
kaggle_json_path = '/kaggle.json'  # Replace with the actual path

# Load the Kaggle credentials
kaggle_credentials = json.load(open(kaggle_json_path))

# setting up the kaggle API key as env var
os.environ['KAGGLE_USERNAME'] = kaggle_credentials['username']
os.environ['KAGGLE_KEY'] = kaggle_credentials['key']

!kaggle datasets download -d abdallahalidev/plantvillage-dataset

# unzip the sownloaded dataset 
with ZipFile('plantvillage-dataset.zip', 'r') as zipObj:
    zipObj.extractall()
print(os.listdir('plantvillage dataset'))

print(len(os.listdir('plantvillage dataset/segmented')))
print(os.listdir('plantvillage dataset/segmented')[:5])

print(len(os.listdir('plantvillage dataset/color')))
print(os.listdir('plantvillage dataset/color')[:5])

print(len(os.listdir('plantvillage dataset/grayscale')))
print(os.listdir('plantvillage dataset/grayscale')[:5])

print(len(os.listdir('plantvillage dataset/color/Grape___healthy')))
print(os.listdir('plantvillage dataset/color/Grape___healthy')[:5])

# Dataset Path
base_dir = "plantvillage dataset/color"
image_path = '/content/plantvillage dataset/color/Apple___Cedar_apple_rust/2770fd33-4612-4c09-80a7-dbac2ec8ff82___FREC_C.Rust 4049.JPG'

# read the img
img = mpimg.imread(image_path)

print(img.shape)
# display the img
plt.imshow(img)
plt.axis('off') # turn off axis numbers
plt.show() 

image_path = '/content/plantvillage dataset/color/Apple___Cedar_apple_rust/2770fd33-4612-4c09-80a7-dbac2ec8ff82___FREC_C.Rust 4049.JPG'

# read the img
img = mpimg.imread(image_path)
print(img)

# img parameters
img_size = 224
batch_size = 32

# img data generators
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
) # use 20% data for validation

# train generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)
# validation generator
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)
# model definition
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(38, activation='softmax'))

# model summary
model.summary()
# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size, # no.s of steps per epoch
    epochs=5, # no.s of epochs
    validation_data = validation_generator,
    validation_steps = validation_generator.samples // batch_size # validation steps
)
# model evaluatoin
print('Evaluating model...')
test_loss, test_acc = model.evaluate(validation_generator, steps = validation_generator.samples//batch_size)
print(f"Test accuracy:, {test_acc*100:.3f}%")

# plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower left')
plt.show()

# plot training and validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower left')

# Fn to load and preprocess the img using PILLOW
def load_and_preprocess_image(image_path, target_size = (224,224)):
    img = Image.open(image_path) # load the img
    img = img.resize(target_size) # resize the img
    # Convert image to RGB if it has an alpha channel
    if img.mode == 'RGBA':
        img = img.convert('RGB')  
    img_array = np.array(img) # convert the img to array
    img_array = np.expand_dims(img_array, axis=0) # add batch dimension
    img_array = img_array.astype('float32')/255.0
    return img_array

# fn to predict the class of an img
def predict_img_class(model, image_path, class_indices):
    preprocessed_array = load_and_preprocess_image(image_path) # Call load_and_preprocess_image to get the preprocessed image array
    prediction = model.predict(preprocessed_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    predicted_class_name = class_indices[predicted_class_index]
    return predicted_class_name
# creating a mapping form class indices to class names
class_indices = {v: k for k, v in train_generator.class_indices.items()}

# class_indices
# saving the class names as json file
json.dump(class_indices, open('class_indices.json', 'w'))

# new example
# image_path = "/Screenshot 2025-05-13 010625.png" # google
# image_path = "/Screenshot 2025-05-13 012800.png" # google
image_path = "/content/plantvillage dataset/color/Corn_(maize)___Northern_Leaf_Blight/00a14441-7a62-4034-bc40-b196aeab2785___RS_NLB 3932.JPG"
predicted_class_name = predict_img_class(model, image_path, class_indices)
print(f"The predicted class is: {predicted_class_name}") # output result

model.save()

'C:\\Users\\naray'